## Merge annotation results with excel sheet of molecular features ##

- - - - -

Folders(for personal reference):

"D:\\JYU_2019_raw data\\Notame_satuaivot(1)\\results\\"    
"D:\\Sanna Lensu_2020\\sanna2020_brain\\results\\"    
"D:\\Sanna Lensu_2020\\sanna2020_feces\\results\\"    
"D:\\Sanna Lensu_2020\\sanna2020_serum\\results\\"

In [1]:
import pandas as pd
import numpy as np
import openpyxl
from openpyxl.utils.dataframe import dataframe_to_rows
import ipywidgets as widgets
import os
import helpers
from difflib import SequenceMatcher


### Type filepath for target excel sheet ##

In [2]:
filename = "results.xlsx"
tgtfolder = "D:\\JYU_2019_raw data\\Notame_satuaivot(1)\\results\\"
tgtfile = "" + tgtfolder + filename

### Type filepath for annotation results ##

In [3]:
resultfile = "D:\\JYU_2019_raw data\\Notame_satuaivot(1)\\results\\MAT\\Structure result-2096.txt"

**Create dataframes**

In [4]:
wb = openpyxl.load_workbook(filename = tgtfile)
sheet = wb.worksheets[0]
metadata = helpers.makeMetadata(sheet)
df_results = helpers.getrExcel(resultfile)
df_target = helpers.getExcel(tgtfile, sheet)
df_data = helpers.getDataExcel(tgtfile, sheet)

**Add a column for Final rank\***

*the sum of ascending anova rank and max-parameter from random forest analysis

In [5]:
df_sorted = df_target.sort_values(by="ANOVA_P_FDR", na_position="last")
df_sorted["finalrank"] = [v + (i+1) for i, v in enumerate(df_sorted["max"])]

**Add columns for metabolite name similiarity score**



In [11]:
merged_sorted_results = pd.concat([df_sorted, df_results], axis=1, sort=False)
merged_sorted_results ["similiarity_1"] = [i for i in map(lambda i: SequenceMatcher(lambda x: x == "nan", str(merged_sorted_results["Structure rank 1"][i]).lower(), str(merged_sorted_results["Metabolite_name"][i]).lower()).ratio(), range(0, merged_sorted_results.shape[0]))]
merged_sorted_results ["similiarity_2"] = [i for i in map(lambda i: SequenceMatcher(lambda x: x == "nan", str(merged_sorted_results["Structure rank 2"][i]).lower(), str(merged_sorted_results["Metabolite_name"][i]).lower()).ratio(), range(0, merged_sorted_results.shape[0]))]
merged_sorted_results ["similiarity_3"] = [i for i in map(lambda i: SequenceMatcher(lambda x: x == "nan", str(merged_sorted_results["Structure rank 3"][i]).lower(), str(merged_sorted_results["Metabolite_name"][i]).lower()).ratio(), range(0, merged_sorted_results.shape[0]))]

**Merge dataframes and write into excel file**

In [12]:
cnumber = merged_sorted_results.shape[1]
merged_df = pd.concat([merged_sorted_results, df_data], axis=1, sort=False)
merged = helpers.mergeSheets(metadata, cnumber, merged_df)

In [13]:
merged.save(tgtfolder + "merge.xlsx")